# Selection of relevant sensors

The first step in application of machine learning methods is to explore and visualize the available data. This means an understanding of the properties and relationships in a dataset (between the data columns and between the data columns and labels).
This task is divided into several stages:
   - Files containing useful data are uploaded from the folder.
   - The size of the data is determined.
   - For each file, it is necessary to check whether the type of the data in columns corresponds to the type that is expected.This also gives an insight if the data is numerical, categorical or mixed.
   - In general cases, it can happen that some cells are empty, or contain NaN values. Because of this, check for empty cells is performed as an example even if it is not expected that this error arise from the sensors' data.
   - Using Pandas method `describe`,some statistical summary is gained.
   - Some basic plots are presented in order to describe the nature of the data from the sensors.

In [125]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os,sys
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import h5py
from scipy import signal

### Files upload, data size and type

Uploading 81 files which correspond to 81 production parts.

In [126]:
scope_traces=[] #this will be the list of dataframes
file_names=os.listdir('Data/AFRC Radial Forge - Zenodoo Upload v3/Data/ScopeTraces')

Upload check

In [127]:
file_names[0]

'Scope0001.csv'

Creating the list of dataframes for the uploaded files:

In [128]:
for file in file_names:
    df=pd.read_csv("Data\\AFRC Radial Forge - Zenodoo Upload v3\\Data\\ScopeTraces\\"+file,encoding = 'unicode_escape') 
    scope_traces.append(df)

In [129]:
type(scope_traces[0]) #checking the type of element in the list (dataframe)

pandas.core.frame.DataFrame

A total number of 81 files (81 sensors' cycles or 81 produced parts per day in the forging process) is analyzed. The first file contains 23328 rows (corresponding to the time samples from 97 sensors). Later, it is going to be discovered that the number of rows differs among all files.

In [130]:
scope_traces[0].head()

,Timer Tick [ms],Block-Nr,Power [kW],Force [kN],A_ges_vibr,Schlagzahl [1/min],EXZ_pos [deg],hydraulic low pressure [bar],hydraulic high pressure [bar],A_ACTpos [mm],B_ACTpos [mm],DB_ACTpos [mm],D_ACTpos [mm],L_ACTpos [mm],R_ACTpos [mm],SBA_ActPos [mm],INDA_ACTpos [deg],A_ACT_Force [kN],B_ACT_Force [kN],DB_ACT_Force [kN],D_ACT_Force [kN],L_ACTspd [mm/min],R_ACTspd [mm/min],SBA_NomPos [mm] [mm],INDA_NOMspd [U/min],A_ACTspd [mm/min],B_ACTspd [mm/min],DB_ACTspd [mm/min],D_ACTspd [mm/min],L_NOMpos [mm],R_NOMpos [mm],SBA_OUT [%],INDA_NOMpos [deg],A_NOMpos [mm],B_NOMpos [mm],DB_NOMpos [mm],D_NOMpos [mm],L_OUT [%],R_OUT [%],Feedback SBA [%],INDA_OUT [%],A_OUT [%],B_OUT [%],DB_OUT [%],D_OUT [%],L_NOMspd [mm/min],R_NOMspd [mm/min],Frc_Volt,INDA_ACTspd [U/min],A_NOMspd [mm/min],B_NOMspd [mm/min],DB_NOMspd [mm/min],D_NOMspd [mm/min],Feedback L [%],Feedback R [%],Speed Vn_1 [rpm],NOMforceSPA [kN],A_NOM_Force [kN],B_NOM_Force [kN],DB_NOM_Force [kN],D_NOM_Force [kN],$F_F41L (I14S8),IP_ActSpd [mm/min],IP_ActPos [mm],SPA_OUT [%],Feedback A [%],Feedback B [%],Feedback DB [%],Feedback D [%],IP_NomSpd [mm/min],IP_NomPos,Feedback_SPA [%],Lub_ActSpd [rpm],RamRetract_ActSpd [rpm],Hyd_ActSpd [rpm],ForgingBox_Temp,TMP_Ind_U1 [°C],TMP_Ind_F [°C],IP_Out [%],ACTforceSPA [kN],$U_GH_HEATON_1 (U25S0),W2 Durchfluss [l],W1 Durchfluss [l],$E_GH_FAULT_2 (I26S21),$E_GH_FAULT_1 (I25S21),$B12R_Y11 (U14S16),L1.R_B41 [bar],O_EMERG,STP || EM,O_MASTOP,c01w,c02w,$U_GH_NOMVAL_1 (U25W1),$H1P_Y12 (U11S17),$H1P_Y11 (U11S7),$U_GH_NOMEXT_2 (U26S1),$U_GH_HEATON_2 (U26S0),$U_GH_NOMEXT_1 (U25S1),$U_GH_HEATON_1 (U25S0).1
0,2594955190,10,50.453915,0.0,7.880536,1207.22081,31.204766,115.256693,202.028443,1207.685,499.950,797.995,700.130,79.921259,79.920872,-0.020,359.928,-4.837847,5.155263,3.801680,1.065573,2.849859,2.037258,0.0,0.0,-0.002999,-3.567227,-7.318537,-60.325765,94.9,94.9,-10.0,37.697855,1200.0,90,350.0,700,3.624816,2.950538,-9.667969,0.0,-7.418207,-5.560455,-4.605909,-5.665290,48.0,48.0,1.015625,0.0,-682.65,-1400.0,-1400.0,0,4.492188,3.393555,0.0,0.813718,0,0,10,5,0,-15.626282,430.078125,50.0,-5.029297,-4.980469,-2.587891,-4.736328,0.0,430,53.442383,1335.0,599.4,799.2,35.874131,243.847656,242.753787,-0.195312,0.813718,0,0.049600,0.024840,0,0,1,35.512785,0,0,0,1,0,0,0,1,1,0,1,0
1,2594955200,10,50.454423,0.0,6.334391,1209.64038,103.758477,115.233426,201.954750,1207.685,499.865,797.925,700.065,79.925348,79.925306,-0.020,359.928,-2.926372,3.306562,2.836382,0.504460,11.354708,11.613530,0.0,0.0,2.131654,-74.906612,-177.361490,-201.706966,94.9,94.9,-10.0,37.697855,1200.0,90,350.0,700,4.212165,3.537096,-9.667969,0.0,-7.615108,-6.594545,-5.332424,-0.844708,60.0,60.0,1.010742,0.0,-682.65,-1750.0,-1750.0,-180,5.224609,4.003906,0.0,0.813718,0,0,10,5,0,-9.476831,430.074219,50.0,-6.738281,-5.834961,-3.320312,-0.830078,0.0,430,53.442383,1335.0,599.4,799.2,35.874370,243.847656,242.784148,-0.185547,0.813718,0,0.049584,0.024779,0,0,1,35.030206,0,0,0,1,0,0,0,1,1,0,1,0
2,2594955210,10,50.453662,0.0,5.486467,1205.96980,176.092461,115.138935,201.898792,1207.680,499.735,797.785,700.005,79.930455,79.931276,-0.025,359.928,9.374414,5.292705,2.518575,0.403512,18.923269,21.136427,0.0,0.0,-0.542491,-186.169649,-435.970590,-269.239445,94.9,94.9,-10.0,37.697855,1200.0,90,350.0,700,4.796142,4.127085,-9.667969,0.0,-7.592444,-7.560556,-6.093333,0.133833,72.0,72.0,1.010742,0.0,-682.20,-2100.0,-2100.0,0,5.957031,4.980469,0.0,0.757770,0,0,10,5,0,-14.923791,430.074219,50.0,-6.738281,-7.055664,-4.296875,1.123047,0.0,430,53.442383,1335.0,599.4,799.2,35.874607,243.847656,242.761722,-0.185547,0.757770,0,0.049493,0.024847,0,0,1,34.880641,0,0,0,1,0,0,0,1,1,0,1,0
3,2594955220,10,50.450142,0.0,114.403358,1210.00307,248.690117,114.985305,201.841733,1207.550,499.530,797.575,699.995,79.937327,79.938360,-0.025,359.928,9.238096,2.319716,2.108604,0.016105,27.658369,29.501334,0.0,0.0,-111.926949,-341.313458,-757.610593,-188.636366,94.9,94.9,-10.0,37.697855,1200.0,90,350.0,700,5.380817,4.716365,-9.667969,0.0,-7.008123,-8.4

In [131]:
scope_traces[0].shape #file size

(23328, 99)

For each dataframe, it is possible to check the type of values in all columns. It is expected that all values are integers *int* or real numbers *float64* because they represent measuring data from the sensors. To demonstrate this step, it will be done on one file.

In [132]:
scope_traces[0].dtypes

Timer Tick [ms]                    int64
Block-Nr                           int64
Power [kW]                       float64
Force [kN]                       float64
A_ges_vibr                       float64
Schlagzahl [1/min]               float64
EXZ_pos [deg]                    float64
hydraulic low pressure [bar]     float64
hydraulic high pressure [bar]    float64
A_ACTpos [mm]                    float64
B_ACTpos [mm]                    float64
DB_ACTpos [mm]                   float64
D_ACTpos [mm]                    float64
L_ACTpos [mm]                    float64
R_ACTpos [mm]                    float64
SBA_ActPos [mm]                  float64
INDA_ACTpos [deg]                float64
A_ACT_Force [kN]                 float64
B_ACT_Force [kN]                 float64
DB_ACT_Force [kN]                float64
D_ACT_Force [kN]                 float64
L_ACTspd [mm/min]                float64
R_ACTspd [mm/min]                float64
SBA_NomPos [mm] [mm]             float64
INDA_NOMspd [U/m

In [149]:
sensors_in_use=[None]*len(scope_traces)
sensors_in_use=[scope_traces[i].drop(columns=['A_NOM_Force [kN]','B_ACTpos [mm]','B_ACT_Force [kN]','B_ACTspd [mm/min]','B_NOMpos [mm]','B_OUT [%]','B_NOMspd [mm/min]','B_NOM_Force [kN]','Feedback B [%]','DB_NOM_Force [kN]','D_ACTpos [mm]','D_ACT_Force [kN]','D_ACTspd [mm/min]','D_NOMpos [mm]','D_OUT [%]','D_NOMspd [mm/min]','D_NOM_Force [kN]','Feedback D [%]','$F_F41L (I14S8)',
'hydraulic low pressure [bar]','hydraulic high pressure [bar]','Lub_ActSpd [rpm]','Hyd_ActSpd [rpm]','$U_GH_HEATON_1 (U25S0)','O_EMERG','STP || EM','O_MASTOP','$U_GH_NOMVAL_1 (U25W1)','$H1P_Y12 (U11S17)','$H1P_Y11 (U11S7)','$U_GH_NOMEXT_2 (U26S1)','$U_GH_HEATON_2 (U26S0)','$U_GH_NOMEXT_1 (U25S1)','$E_GH_FAULT_2 (I26S21)','$E_GH_FAULT_1 (I25S21)','$B12R_Y11 (U14S16)','Timer Tick [ms]','Block-Nr']) for i in range(len(scope_traces))]


In [134]:
def plot_sensors(part):
    column_names=part.columns
    for column in column_names:
        fig = plt.figure(figsize=(4,4)) # define plot area
        plt.plot(np.arange(0,(part.shape[0]/100),0.01),part[column].values)
        plt.xlabel("Time, s")
        plt.ylabel(column)
        plt.show()

In [135]:
pd.options.display.max_columns = 4000

In [136]:
from ipywidgets import interact
import ipywidgets as widgets

In [137]:
def df_summary(i):
    print("Part",sensors_in_use[i].describe())
interact(df_summary,i=widgets.IntSlider(min=0, max=81, step=1))  


interactive(children=(IntSlider(value=0, description='i', max=81), Output()), _dom_classes=('widget-interact',…

<function __main__.df_summary(i)>

In [138]:
def calculate_correlation(sensors):
    correlation_list=[sensors[i].corr().values for i in range(len(sensors))]
    correlation_array=np.array(correlation_list)
    print("Correlation array dimensions:",correlation_array.shape)
    corr_mean=np.mean(correlation_array,axis=0)
    print("Mean value of correlation coefficients for all sensors:")
    corr_mean_df=pd.DataFrame(corr_mean,columns=sensors[0].columns,index=sensors[0].columns)
    return corr_mean_df.style.background_gradient(cmap='coolwarm').set_precision(2)

In [139]:
calculate_correlation(sensors_in_use)

Correlation array dimensions: (81, 61, 61)
Mean value of correlation coefficients for all sensors:


In [150]:
sensors_in_use=[sensors_in_use[i].drop(columns=['L_ACTspd [mm/min]','L_NOMpos [mm]','L_OUT [%]','L_NOMspd [mm/min]','Feedback L [%]' ]) for i in range(len(scope_traces))]

In [141]:
def plot_sensors(i):
    column_names=sensors_in_use[i].columns
    for column in column_names:
        fig = plt.figure(figsize=(4,4)) # define plot area
        plt.plot(np.arange(0,(sensors_in_use[i].shape[0]/100),0.01),sensors_in_use[i][column].values)
        plt.xlabel("Time, s")
        plt.ylabel(column)
        plt.show()
interact(plot_sensors,i=widgets.IntSlider(min=0, max=80, step=1))  

interactive(children=(IntSlider(value=0, description='i', max=80), Output()), _dom_classes=('widget-interact',…

<function __main__.plot_sensors(i)>

In [151]:
sensors_in_use=[sensors_in_use[i].drop(columns=['EXZ_pos [deg]','L_ACTpos [mm]','IP_ActSpd [mm/min]','IP_NomSpd [mm/min]','RamRetract_ActSpd [rpm]','IP_Out [%]','L1.R_B41 [bar]',
'ForgingBox_Temp','Schlagzahl [1/min]']) for i in range(len(sensors_in_use))]

In [143]:
sensors_in_use[0].shape

(23328, 47)

In [144]:
calculate_correlation(sensors_in_use)

Correlation array dimensions: (81, 47, 47)
Mean value of correlation coefficients for all sensors:


In [152]:
sensors_in_use=[sensors_in_use[i].drop(columns=['Power [kW]','INDA_NOMspd [U/min]','R_NOMpos [mm]','INDA_NOMpos [deg]','A_NOMpos [mm]','INDA_OUT [%]','INDA_ACTspd [U/min]',
                                       'NOMforceSPA [kN]','ACTforceSPA [kN]','W2 Durchfluss [l]','SBA_ActPos [mm]','DB_NOMpos [mm]','R_OUT [%]','R_NOMspd [mm/min]','Feedback R [%]',
                                               'A_OUT [%]','A_NOMspd [mm/min]','Feedback A [%]','DB_OUT [%]','DB_NOMspd [mm/min]','Feedback DB [%]','SBA_OUT [%]','IP_NomPos','Feedback_SPA [%]','ACTforceSPA [kN]']) for i in range(len(sensors_in_use))]

In [153]:
sensors_in_use[0].shape

(23328, 23)

In [154]:
def plot_sensors(i):
    column_names=sensors_in_use[i].columns
    for column in column_names:
        fig = plt.figure(figsize=(4,4)) # define plot area
        plt.plot(np.arange(0,(sensors_in_use[i].shape[0]/100),0.01),sensors_in_use[i][column].values)
        plt.xlabel("Time, s")
        plt.ylabel(column)
        plt.show()
interact(plot_sensors,i=widgets.IntSlider(min=0, max=80, step=1))  

interactive(children=(IntSlider(value=0, description='i', max=80), Output()), _dom_classes=('widget-interact',…

<function __main__.plot_sensors(i)>

In [155]:
calculate_correlation(sensors_in_use)

Correlation array dimensions: (81, 23, 23)
Mean value of correlation coefficients for all sensors:


,Force [kN],A_ges_vibr,A_ACTpos [mm],DB_ACTpos [mm],R_ACTpos [mm],INDA_ACTpos [deg],A_ACT_Force [kN],DB_ACT_Force [kN],R_ACTspd [mm/min],SBA_NomPos [mm] [mm],A_ACTspd [mm/min],DB_ACTspd [mm/min],Feedback SBA [%],Frc_Volt,Speed Vn_1 [rpm],IP_ActPos [mm],SPA_OUT [%],TMP_Ind_U1 [°C],TMP_Ind_F [°C],W1 Durchfluss [l],c01w,c02w,$U_GH_HEATON_1 (U25S0).1
Force [kN],1,0.25,-0.78,-0.28,-0.65,0.53,0.28,0.089,-0.41,-0.00036,0.043,0.0029,0.0012,0.44,0.57,0.54,-0.64,-0.52,-0.062,0.85,-0.63,-0.07,-0.45
A_ges_vibr,0.25,1,-0.31,0.019,-0.25,0.26,0.13,-0.014,-0.032,0.0008,0.0057,0.022,0.00014,0.12,0.26,0.2,-0.33,-0.25,0.2,0.19,-0.34,0.015,-0.17
A_ACTpos [mm],-0.78,-0.31,1,0.042,0.81,-0.72,-0.26,0.072,0.38,5.8e-05,0.0096,-0.11,-0.00098,-0.37,-0.76,-0.51,0.84,0.67,-0.12,-0.59,0.83,0.13,0.44
DB_ACTpos [mm],-0.28,0.019,0.042,1,-0.27,0.029,-0.063,-0.16,0.5,0.00018,0.19,-0.002,-0.00056,-0.043,0.012,-0.31,-0.17,-0.023,0.39,-0.28,-0.25,0.63,0.33
R_ACTpos [mm],-0.65,-0.25,0.81,-0.27,1,-0.62,-0.059,0.13,0.002,8.9e-05,-0.22,-0.17,-0.0016,-0.34,-0.65,-0.29,0.77,0.56,-0.25,-0.52,0.77,0.018,0.25
INDA_ACTpos [deg],0.53,0.26,-0.72,0.029,-0.62,1,0.16,-0.08,-0.2,-5e-05,0.051,0.082,0.00072,0.25,0.56,0.35,-0.62,-0.49,0.16,0.39,-0.62,-0.078,-0.3
A_ACT_Force [kN],0.28,0.13,-0.26,-0.063,-0.059,0.16,1,0.078,-0.29,-0.24,-0.42,-0.02,-0.23,0.091,0.08,0.24,-0.24,-0.23,-0.076,0.28,-0.23,0.07,-0.22
DB_ACT_Force [kN],0.089,-0.014,0.072,-0.16,0.13,-0.08,0.078,1,-0.18,0.075,-0.0015,-0.17,0.17,-0.049,0.0066,0.0069,0.11,0.084,-0.17,0.27,0.13,-0.13,0.00045
R_ACTspd [mm/min],-0.41,-0.032,0.38,0.5,0.002,-0.2,-0.29,-0.18,1,0.001,0.26,0.013,0.007,-0.14,-0.24,-0.28,0.037,0.19,0.37,-0.34,-0.00012,0.4,0.26
SBA_NomPos [mm] [mm],-0.00036,0.0008,5.8e-05,0.00018,8.9e-05,-5e-05,-0.24,0.075,0.001,1,-0.0045,0.018,0.37,0.053,0.088,-8.1e-05,0.0011,6.3e-05,2.4e-05,-0.00016,7.5e-05,3.5e-05,8.4e-05


In [158]:
forging_sensors=[None]*len(sensors_in_use)
forging_sensors=[sensors_in_use[i][['Force [kN]','A_ges_vibr','A_ACTpos [mm]', 'DB_ACTpos [mm]',
       'R_ACTpos [mm]', 'INDA_ACTpos [deg]', 'A_ACT_Force [kN]',
       'DB_ACT_Force [kN]', 'R_ACTspd [mm/min]', 'SBA_NomPos [mm] [mm]',
       'A_ACTspd [mm/min]', 'DB_ACTspd [mm/min]', 'Feedback SBA [%]',
       'Frc_Volt', 'Speed Vn_1 [rpm]', 'IP_ActPos [mm]', 'SPA_OUT [%]','TMP_Ind_F [°C]', 'W1 Durchfluss [l]', 'c01w','c02w']] for i in range(len(sensors_in_use))]

In [160]:
heating_sensor=[None]*len(sensors_in_use)
heating_sensor=[sensors_in_use[i][['TMP_Ind_U1 [°C]']] for i in range(len(sensors_in_use))]

In [ ]:
#STOP

### Check for missing values

Function *check_empty* checks whether the file contains empty cells. Here, it is supposed that empty cell is a blank cell (" ").

In [ ]:
def check_empty(trace):
    empty_cell=(trace.astype(np.object) == '').any()
    [print("Empty_cell") for x in empty_cell if x==True]

In [ ]:
for i in range(len(scope_traces)): #check for all of 81 files
    print(i, end="\r") 
    check_empty(scope_traces[i])

### Statistical summary

Based on previous notebooks and similar plots where changes in signals and peaks were observed, sensors are divided into two groups: forging and heating for each produced part. These sensors were chosen based on their importance for the heating or forging phases. It is important to notice that some sensors were dropped because they are not in use, or because they are described as auxiliary sensors. 

In [ ]:
forging_sensors=np.zeros((len(scope_traces), *scope_traces[0].shape))

forging_sensors=[scope_traces[part][['Power [kW]', 'Force [kN]', 'A_ges_vibr','Schlagzahl [1/min]', 'RamRetract_ActSpd [rpm]',
       'A_ACTpos [mm]', 'L_ACTpos [mm]', 'R_ACTpos [mm]','SBA_ActPos [mm]', 'A_ACT_Force [kN]', 'DB_ACT_Force [kN]',
       'L_NOMpos [mm]', 'R_NOMpos [mm]', 'INDA_NOMpos [deg]','A_NOMpos [mm]', 'Frc_Volt','ForgingBox_Temp', 'L1.R_B41 [bar]', 'TMP_Ind_F [°C]','W2 Durchfluss [l]', 'W1 Durchfluss [l]']] for part in range(len(scope_traces))] 

In [ ]:
heating_sensors=np.zeros((len(scope_traces), *scope_traces[0].shape))
heating_sensors=[scope_traces[part][['TMP_Ind_U1 [°C]','IP_ActPos [mm]', 'IP_NomPos']] for part in range(len(scope_traces))]

Statistical overview of the sensors is made by function *df_summary*. This is demonstrated for the first production part only, but if this function runs for other parts, the difference in sensors' cycles (duration of the production of each part) can be seen by the `count`property. It can be noticed that for some sensors standard deviation from the mean value is very high. Also, the median value much differs from the mean value in some cases. This situation implies that it might be useful to extract the heating and forging phase instead of analyzing the signals for the whole production process. 

In [ ]:
def df_summary(part):
    print("Part", part.describe())

In [ ]:
df_summary(forging_sensors[0])

In [ ]:
df_summary(heating_sensors[0])

The statistical summary can also help to eliminate the sensors with **low variance** and **zero variance**. Zero variance features are comprised of the same values. Low variance features arise from features with most values the same and with few unique values.  
In addition to the statistical summary, function *plot_sensors* plots signals from the sensor recorded during the whole production process of one part (comprised of three phases: heating, transferring and forging).

In [ ]:
def plot_sensors(part):
    column_names=part.columns
    for column in column_names:
        fig = plt.figure(figsize=(4,4)) # define plot area
        plt.plot(np.arange(0,(part.shape[0]/100),0.01),part[column].values)
        plt.xlabel("Time, s")
        plt.ylabel(column)
        plt.show()

In [ ]:
plot_sensors(forging_sensors[0])

In [ ]:
plot_sensors(heating_sensors[0])

Pearson correlation coefficients between different sensors are calculated in the next cell. Correlation can have big impact on feature selection. Sensors with big correlation should be detected and they should not be in the same Machine Learning model.Correlation coefficients are calculated as the mean values for 81 parts.

In [ ]:
def calculate_correlation(sensors):
    correlation_list=[sensors[i].corr().values for i in range(len(sensors))]
    correlation_array=np.array(correlation_list)
    print("Correlation array dimensions:",correlation_array.shape)
    corr_mean=np.mean(correlation_array,axis=0)
    print("Mean value of correlation coefficients for all sensors:")
    corr_mean_df=pd.DataFrame(corr_mean,columns=sensors[0].columns,index=sensors[0].columns)
    return corr_mean_df.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
calculate_correlation(forging_sensors) #mean values of corr coefficients for all parts

In [ ]:
calculate_correlation(heating_sensors) #mean values of corr coefficients for all parts

## Extraction of the heating phase and forging phase from the sensors' signals

The following analysis is based on the extraction of the heating and forging phase from the sensors. Then, all of the previous steps are repeated. 

Merge data from all parts into one Dataframe:

In [ ]:
for index,df in enumerate(scope_traces):
    df['traceID'] = index+1

In [ ]:
merged_data = pd.concat(scope_traces, ignore_index=True)
merged_data['Time [s]']=(merged_data.index.values)/100.0

Function *extract_heating_phases* uses the value of the sensor "$U_GH_HEATON_1 (U25S0).1" greater than zero as reference for the start of the heating phase. When the value becomes zero, it means that heating phase is finished. This phase is extracted only for heating sensors.

In [ ]:
def extract_heating_phases(trace):
    heating_start=[]
    heating_stop=[]
    
    digital_sig_heating = trace['$U_GH_HEATON_1 (U25S0).1']>0
    heating_diff = digital_sig_heating.astype('int').diff()
    heating_start.append(heating_diff[heating_diff==1].index.values) # difference between next cell and previous cell 
                                                                    #is positive because the heating phase is still active
    heating_stop.append(heating_diff[heating_diff==-1].index.values) # difference between next cell and previous cell 
                                                                    #is negative because the heating phase is finished
    heat_list=list(zip(heating_start[0], heating_stop[0]))
    heating_ph=[None]*(len(heat_list))
                  
    heating_ph=[trace[['TMP_Ind_U1 [°C]','IP_ActPos [mm]','IP_NomPos']][heat_list[i][0]:heat_list[i][1]] for i in range(len(heat_list))] 
        
    return heating_ph


Function *extract_forging_phases* uses the value of the sensor "Force [kN]" greater than zero as reference for the start of the forging phase.

In [ ]:
def extract_forging_phases(trace):
    forging_start=[]
    forging_end=[]
    digital_sig_forge = trace['Force [kN]']>0
    forge_diff = digital_sig_forge.astype('int').diff()
    forging_start.append(forge_diff[forge_diff==1].index.values)
    forging_end.append(forge_diff[forge_diff==-1].index.values) 
                    
    forge_list=list(zip(forging_start[0], forging_end[0]))     
    forging_ph=[None]*(len(forge_list))
    forging_ph=[trace[['Power [kW]', 'Force [kN]', 'A_ges_vibr','Schlagzahl [1/min]', 'RamRetract_ActSpd [rpm]',
       'A_ACTpos [mm]', 'L_ACTpos [mm]', 'R_ACTpos [mm]','SBA_ActPos [mm]', 'A_ACT_Force [kN]', 'DB_ACT_Force [kN]',
       'L_NOMpos [mm]', 'R_NOMpos [mm]', 'INDA_NOMpos [deg]','A_NOMpos [mm]', 'Frc_Volt','ForgingBox_Temp', 'L1.R_B41 [bar]', 'TMP_Ind_F [°C]','W2 Durchfluss [l]', 'W1 Durchfluss [l]']][forge_list[i][0]:forge_list[i][1]] for i in range(len(forge_list))]
    return forging_ph

In [ ]:
heating_ph=extract_heating_phases(merged_data)

In [ ]:
forging_ph=extract_forging_phases(merged_data)

Minimum duration of heating and forging phase for all parts that were produced:

In [ ]:
cycle_length_heat=[None]*(len(heating_ph))
cycle_length_forge=[None]*(len(heating_ph))
cycle_length_heat=[(heating_ph[i].shape[0]) for i in range(len(heating_ph))]
cycle_length_forge=[(forging_ph[i].shape[0]) for i in range(len(heating_ph))]
min_length_heat=min(cycle_length_heat)
min_length_forge=min(cycle_length_forge)
print("Minimum length of time signals  for the heating phase is :",min_length_heat, "and it is in",cycle_length_heat.index(min_length_heat),". cycle")
print("Minimum length of time signals  for the forging phase is :",min_length_forge, "and it is in",cycle_length_forge.index(min_length_forge),". cycle")    

Because of the different length of heating phase and forging phase of time signals, the minimum duration for both phases is extracted. Then, steps for the calculation of statistical summary and correlation coefficients are repeated.

In [ ]:
heating_ph_extracted=[None]*(len(heating_ph))
heating_ph_extracted=[heating_ph[x].iloc[:10997,:] for x in range(len(heating_ph))]

In [ ]:
forging_ph_extracted=[None]*(len(heating_ph))
forging_ph_extracted=[forging_ph[x].iloc[:5619,:] for x in range(len(forging_ph))]

In [ ]:
df_summary(heating_ph_extracted[10])

From the statistical summary for the heating phase, it can be seen that standard deviation for two of these sensors is close to zero. Because of this, these sensors are eliminated from the future observations.

In [ ]:
heating_sensors=[heating_ph_extracted[i].drop(columns=['IP_NomPos',"IP_ActPos [mm]"]) for i in range(len(heating_ph_extracted))]

In [ ]:
df_summary(forging_ph_extracted[0])

Standard deviation for the sensors: L1.R_B41 [bar], ForgingBox_Temp are close to zero. They will be eliminated.  

In [ ]:
#strong correlation
forging_sensors=[forging_ph_extracted[i].drop(columns=['L1.R_B41 [bar]',"ForgingBox_Temp"]) for i in range(len(forging_ph_extracted))]

In [ ]:
calculate_correlation(forging_sensors) #mean values of corr coefficients for all parts

In [ ]:
#strong correlation
forging_sensors=[forging_ph_extracted[i].drop(columns=['Power [kW]','R_NOMpos [mm]',"L_NOMpos [mm]", "R_ACTpos [mm]",'W1 Durchfluss [l]',"A_NOMpos [mm]","L_NOMpos [mm]",'L1.R_B41 [bar]',"ForgingBox_Temp"]) for i in range(len(forging_ph_extracted))]

In [ ]:
calculate_correlation(forging_sensors)

In [ ]:
plot_sensors(forging_sensors[0])

In [ ]:
forging_sensors[0].shape

In [ ]:
heating_sensors[0].shape

In [ ]:
len(heating_sensors)

In [ ]:
plot_sensors(heating_sensors[0])

Now, data is going to be arranged in a 3D matrix, in a way that first dimension represents number of sensors, the second - number of cycles (81 cycles for 81 part produced) and the third one represents number of discrete points (measurement values).

In [ ]:
#split in dimensions 13x81x5619:
forging_sensors=[np.transpose(forging_sensors[i])for i in range(len(forging_sensors))] #81x14x5619
#define the array of sensors (13x81x5619):
split_forging_sensors=np.zeros((forging_sensors[0].shape[0],len(forging_sensors),forging_sensors[0].shape[1]))
#turn 81x13 into 13x81
for i in range(forging_sensors[0].shape[0]):
    for j in range(len(forging_sensors)):
        xx=np.asarray(forging_sensors[j])
        split_forging_sensors[i,j]=xx[i]
print("Dimensions of matrix:", split_forging_sensors.shape)

In [ ]:
#split in dimensions 2x81x10997:
heating_sensors=[np.transpose(heating_sensors[i])for i in range(len(heating_sensors))] #81x2x10997
#define the array of sensors (2x81x10997):
split_heating_sensors=np.zeros((heating_sensors[0].shape[0],len(heating_sensors),heating_sensors[0].shape[1]))
#turn 81x2 into 2x81
for i in range(heating_sensors[0].shape[0]):
    for j in range(len(heating_sensors)):
        xx=np.asarray(heating_sensors[j])
        split_heating_sensors[i,j]=xx[i]
print("Dimensions of matrix:", split_heating_sensors.shape)

In [ ]:
def plot_density_hist_forge(num_of_sensor,time):
    sns.set_style("whitegrid")
    sns.distplot(split_forging_sensors[num_of_sensor,:,time], bins = 5, rug=True, hist = True)
     # Give the plot a main title
         # Set text for the x axis
    plt.title("Histogram of"+" "+forging_sensors[0].index[num_of_sensor])
    plt.show()
interact(plot_density_hist_forge,num_of_sensor=widgets.IntSlider(min=0, max=14, step=1),time=widgets.IntSlider(min=0, max=5619, step=1))

In [ ]:
def plot_density_hist_heat(num_of_sensor,time):
    sns.set_style("whitegrid")
    sns.distplot(split_heating_sensors[num_of_sensor,:,time], bins = 5, rug=True, hist = True)
     # Give the plot a main title
         # Set text for the x axis
    plt.title("Histogram of"+" "+heating_sensors[0].index[num_of_sensor])
    plt.show()
interact(plot_density_hist_heat,num_of_sensor=widgets.IntSlider(min=0, max=2, step=1),time=widgets.IntSlider(min=0, max=10997, step=1))


For each sensor, 20 s interval around midpoint is extracted and will be used in the next steps.

In [ ]:
hf_forge = h5py.File('forge_sensors.h5', 'w')
hf_heat=h5py.File('heating_sensors.h5', 'w')

These sensors are saved separately for the use in the following notebooks.

In [ ]:
[hf_forge.create_dataset('Sensor'+str(i),data=split_forging_sensors[i][:,1809:3809]) for i in range(len(split_forging_sensors))]

In [ ]:
[hf_heat.create_dataset('Sensor'+str(i),data=split_heating_sensors[i][:,4498:6498]) for i in range(len(split_heating_sensors))]

In [ ]:
hf_forge.close()

In [ ]:
hf_heat.close()

Now, a white noise $\epsilon$ is added to the observed part of the signals (20 s):

$$x_{n}(t) = x(t)+\epsilon$$


A white noise is defined with normal distribution - 

White noise has normal distribution ${\mathcal {N}}(0 ,\sigma ^{2})$, where $\sigma ^{2}$ represents variance. The process of adding a white noise is as follows:
* Step1: For each sensor, a mean value for the centered part of the signals is calculated. 
* Step2: Standard deviation $\sigma$ of a white noise is defined as 1% of the mean value calculated in the Step 1.
This has been done separately for forging and heating sensors.

In [ ]:
mean_value_forge= [split_forging_sensors[i][:,1809:3809].mean() for i in range(len(split_forging_sensors))]
white_noise_forge=[np.random.randn(2000)*0.01*(mean_value_forge[i]) for i in range(len(split_forging_sensors))]

mean_value_heat= [split_heating_sensors[i][:,4498:6498].mean() for i in range(len(split_heating_sensors))]
white_noise_heat=[np.random.randn(2000)*0.01*(mean_value_heat[i]) for i in range(len(split_heating_sensors))]

White noise signal is plotted together with original signal to observe if the value of variance can be correctly used for each sensor.

In [ ]:
def plot_white_noise_forge(num_of_sensor,part):
    
    fig = plt.figure(figsize=(7,7)) # define plot area
    plt.plot(np.arange(0,20,0.01),white_noise_forge[num_of_sensor],label="white noise")
    plt.plot(np.arange(0,20,0.01),split_forging_sensors[num_of_sensor][part,1809:3809], label="original signal")
    plt.xlabel("Time, s")
    plt.ylabel(forging_sensors[0].index[num_of_sensor])
    plt.legend()
    plt.show()
interact(plot_white_noise_forge,num_of_sensor=widgets.IntSlider(min=0, max=14, step=1),part=widgets.IntSlider(min=0, max=81, step=1))


In [ ]:
def plot_white_noise_heat(num_of_sensor,part):
    
    fig = plt.figure(figsize=(7,7)) # define plot area
    plt.plot(np.arange(0,20,0.01),white_noise_heat[num_of_sensor],label="white noise")
    plt.plot(np.arange(0,20,0.01),split_heating_sensors[num_of_sensor][part,4498:6498], label="original signal")
    plt.xlabel("Time, s")
    plt.ylabel(heating_sensors[0].index[num_of_sensor])
    plt.legend()
    plt.show()
interact(plot_white_noise_heat,num_of_sensor=widgets.IntSlider(min=0, max=2, step=1),part=widgets.IntSlider(min=0, max=81, step=1))


After an insight through plots, white noise signals are added to the original signals from sensors.

In [ ]:
#adding white noise
def add_white_noise(sensors,white_noise,start,end):
    sensors_with_noise=[sensors[i][:,start:end]+ white_noise[i] for i in range(len(sensors))]   
    return sensors_with_noise

In [ ]:
sensors_with_noise_forge=add_white_noise(split_forging_sensors,white_noise_forge,1809,3809)
sensors_with_noise_heat=add_white_noise(split_heating_sensors,white_noise_heat,4498,6498)

In [ ]:
def plot_signal_with_noise_forge(num_of_sensor,part):
    fig = plt.figure(figsize=(7,7)) # define plot area
    plt.plot(np.arange(0,20,0.01),sensors_with_noise_forge[num_of_sensor][part],label=" Signal with white noise")
    plt.plot(np.arange(0,20,0.01),split_forging_sensors[num_of_sensor][part,1809:3809], label="Original signal")
    plt.xlabel("Time, s")
    plt.ylabel(forging_sensors[0].index[num_of_sensor])
    plt.legend()
    plt.show()
interact(plot_signal_with_noise_forge,num_of_sensor=widgets.IntSlider(min=0, max=14, step=1),part=widgets.IntSlider(min=0, max=81, step=1))


In [ ]:
def plot_signal_with_noise_heat(num_of_sensor,part):
    fig = plt.figure(figsize=(7,7)) # define plot area
    plt.plot(np.arange(0,20,0.01),sensors_with_noise_heat[num_of_sensor][part],label="Signal with white noise")
    plt.plot(np.arange(0,20,0.01),split_heating_sensors[num_of_sensor][part,4498:6498], label="Original signal")
    plt.xlabel("Time, s")
    plt.ylabel(heating_sensors[0].index[num_of_sensor])
    plt.legend()
    plt.show()
interact(plot_signal_with_noise_heat,num_of_sensor=widgets.IntSlider(min=0, max=2, step=1),part=widgets.IntSlider(min=0, max=81, step=1))


In [ ]:
forging_sensors[0].index

In [ ]:
time=20
n_of_sampling_points=2000
freq = np.fft.rfftfreq(n_of_sampling_points, float(time)/n_of_sampling_points)   # frequency axis
amp = np.fft.rfft(sensors_with_noise_forge[0][0])   

# END

In [ ]:
m=sensors_with_noise_forge[0][0].size

In [ ]:
w = np.fft.rfft(sensors_with_noise_forge[0][0] * signal.get_window("hamming", m))

In [ ]:
plt.plot(np.arange(2000),sensors_with_noise_forge[0][0] * signal.get_window("hamming", m))

In [ ]:
time=20
n_of_sampling_points=2000
freq = np.fft.rfftfreq(n_of_sampling_points, float(time)/n_of_sampling_points)   # frequency axis

In [ ]:
fig = plt.figure(figsize=(15,5))
plt.plot(freq, 20*np.log10(np.abs(w)))
plt.xlabel("Frequency (Hz)")  
plt.ylabel("Amplitude (Pa)")  

In [ ]:
fig = plt.figure(figsize=(15,5))

plt.plot(freq, (np.abs(w)))
plt.xlabel("Frequency (Hz)")  
plt.ylabel("Amplitude (Pa)")  

In [ ]:
plt.plot(np.arange(0,20,0.01),sensors_with_noise_forge[0][0], label="Real values")
plt.ylabel("Microphone (Pa)")
plt.xlabel("Time (s)")

In [ ]:
fig = plt.figure(figsize=(15,5))
plt.plot(freq,np.abs(amp))
plt.xlabel("Frequency (Hz)")  
plt.ylabel("Amplitude (Pa)")  

In [ ]:
time=20
n_of_sampling_points=2000
freq = np.fft.rfftfreq(n_of_sampling_points, float(time)/n_of_sampling_points)   # frequency axis
amp = np.fft.rfft(sensors_with_noise_heat[0][0])   

In [ ]:
plt.plot(np.arange(0,20,0.01),sensors_with_noise_heat[0][0], label="Real values")
plt.ylabel("Microphone (Pa)")
plt.xlabel("Time (s)")

In [ ]:
fig = plt.figure(figsize=(15,5))
plt.plot(freq,np.abs(amp))
plt.xlabel("Frequency (Hz)")  
plt.ylabel("Amplitude (Pa)")  